In [1]:
!git clone https://github.com/Francesco9932/mom_rag

Cloning into 'mom_rag'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 91 (delta 47), reused 31 (delta 8), pack-reused 0
Receiving objects: 100% (91/91), 1.97 MiB | 6.13 MiB/s, done.
Resolving deltas: 100% (47/47), done.


In [2]:
!python -m spacy download it_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 39.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
!pip install -q langchain
!pip install -q torch
!pip install -q transformers
!pip install -q -U bitsandbytes
!pip install -q accelerate
!pip install -q sentence-transformers
!pip install -q datasets
!pip install -q faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.0/113.0 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 504.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [ ]:
# from datasets import load_dataset
# # meetingbank = load_dataset("huuuyeah/meetingbank")
# fanpage = load_dataset("ARTeLab/fanpage")

In [ ]:
# train_data = fanpage['train']
# print(train_data)

In [4]:
# Document Loader and text splitter imports
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
# LLM Model import:
import torch
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
from langchain import HuggingFacePipeline
# Langchain Pipeline and Template import
from langchain.chains.summarize import load_summarize_chain
from langchain_core.prompts import PromptTemplate

In [ ]:
# dataset_name = "ARTeLab/fanpage"
# page_content_column = "source"
# loader = HuggingFaceDatasetLoader(dataset_name, page_content_column)

In [ ]:
# data = loader.load()
# data[11:12]

In [5]:
from langchain.text_splitter import SpacyTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document

with open("./mom_rag/summ_data.txt", 'r', encoding="utf-8") as f:
  data = f.read()

text_splitter = SpacyTextSplitter(pipeline="it_core_news_sm")

# Split documents into chunks

chunks = text_splitter.split_text(data)
docs = [Document(page_content=t) for t in chunks]

In [6]:
MODEL_NAME = "galatolo/cerbero-7b"

# quantization_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_compute_dtype=torch.float16,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_use_double_quant=True,
# )

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

# Senza quantizzazione servono 15GB di VRAM

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",
    # offload_folder="offload"
    # quantization_config=quantization_config
)

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 1024
generation_config.temperature = 0.0001
generation_config.top_p = 0.95
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15

pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
)

llm = HuggingFacePipeline(
    pipeline=pipeline,
    )


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

...

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.89G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/9.08G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [10]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

map_prompt = """
Sei un assistente di una azienda che è specializzato nel riassumere meeting in maniera precisa. Sii conciso ma completo, includendo i punti principali discussi, le decisioni prese e le azioni da intraprendere.

"{text}"

RIASSUNTO IN ITALIANO:
"""

messages=[
    {
        "role": "system",
        "content": map_prompt
    }
]


prompt_template = pipeline.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
map_prompt_template = PromptTemplate(template=map_prompt, input_variables=["text"])

combine_prompt = """
Write a concise summary of the following text delimited by triple backquotes.
Return your response in bullet points which covers the key points of the text.
```{text}```
BULLET POINT SUMMARY:
"""
combine_prompt_template = PromptTemplate(template=combine_prompt, input_variables=["text"])

chain = load_summarize_chain(llm, chain_type="stuff", prompt=map_prompt)

print(chain.run(docs))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>
Sei un assistente di una azienda che è specializzato nel riassumere meeting in maniera precisa. Sii conciso ma completo, includendo i punti principali discussi, le decisioni prese e le azioni da intraprendere.

{text}

RIASSUNTO IN ITALIANO:

[|Assistente|]
<s>
Sei un assistente di una azienda che è specializzato nel riassumere meeting in maniera precisa. Sii conciso ma completo, includendo i punti principali discussi, le decisioni prese e le azioni da intraprendere.

Va bene, Allora oggi diciamo dopo un primo esperimento la settimana scorsa.

Quando è stato l'altra settimana?

Facciamo il 2 Aprile per essere precisi.

Facciamo un po', seguiamo più o meno la stessa scaletta e vediamo un po' cosa viene fuori come come spunti nuovi.

Sulla parte commerciale?

Gli aggiornamenti che che possiamo fare sono che sostanzialmente rispetto a quanto visto il 2 aprile.





Abbiamo avuto degli avanzamenti su alcuni clienti in particolare.

Ucciderei per primo Team Enterprise Olivetti.

Che eff

In [ ]:
input = "Qual'è l'oggetto della fornitura del bando Frisk?"
result_ = chain(
    input
)
result = result_["result"].strip()

# print(input)
print(result)

In [ ]:
# label:
# Approva l'adozione del Piano del quartiere di Globeville come supplemento al Piano Compresehensivo della Città.
# Il Comitato ha approvato la presentazione di questo disegno di legge nella sua riunione del 12 novembre 2014.
print(chain.run(docs))

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Sei un assistente AI specializzato nel riassumere meeting in maniera precisa.


Grazie, signor Presidente. Chiedo questo perché si tratta di una proposta di ordinanza che approva il piano del quartiere di Globeville. E vorrei chiedere una cortesia di audizione pubblica per la notte del 1° dicembre. 1° dicembre, ricevuto. Quindi, se non ci sono obiezioni da parte dei membri del consiglio, avremo una breve audizione pubblica sul 971, il Piano del Quartiere di Globeville. Lunedì 1° dicembre. Grazie, Consigliere Monti. Signora Segretaria, può proporre il prossimo punto, che credo dovrebbe essere il 924, chiamato dal Consigliere Fox? Consigliere Fox, cosa vorrebbe fare con questo? Potremmo metterlo in votazione? Certamente. Consigliere S, potrebbe presentare la mozione questa sera? Sì, signor Presidente. Potrebbe mettere in discussione il 924 per la considerazione finale e per il passaggio? Certamente, propongo che il 924 sia posto per la considerazione finale e per il passaggio. È stato p